In [62]:
import sys, os
import pandas as pd
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

from Clust.clust.ML.common import ML_pipeline, tool
app_name= "Hs2SwineFarmWithWeatherTime" # "Hs2SwineFarmWithWeatherTime", "energy"

cpu is available.


## 1. Data Preparation

### 1-1. Parameter Setting

In [63]:
if app_name == "energy":
    model_purpose = 'regression'
    feature_X_list = ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    feature_y_list = ['value']
    split_mode ="windows_split"
    data_y_flag = True # 이미 만들어진 Y 데이터를 활용함
    
elif app_name == "Hs2SwineFarmWithWeatherTime":
    model_purpose = 'forecasting' 
    feature_X_list = ['Temperature', 'out_temp','sin_hour']
    feature_y_list = ['Temperature']
    split_mode = 'step_split'
    data_y_flag = False # Y데이터는 없음, X 에서 Y 데이터를 도출함
    
step = 'train'
bucket_name = 'integration' 
data_clean_level = 4

In [64]:
all_integrated_ms_list = db_client.measurement_list(bucket_name)
print(all_integrated_ms_list)
print("==========================================================")
collection_list = mongo_client.get_collection_list(bucket_name)
print(collection_list)

['forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_testX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_trainX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX', 'forecasting_strawberryOpen_cleanLevel0_testX', 'forecasting_strawberryOpen_cleanLevel0_trainX', 'forecasting_strawberryOpen_cleanLevel4_testX', 'forecasting_strawberryOpen_cleanLevel4_trainX', 'regression_energy_cleanLevel0_testX', 'regression_energy_cleanLevel0_testy', 'regression_energy_cleanLevel0_trainX', 'regression_energy_cleanLevel0_trainy', 'regression_energy_cleanLevel4_testX', 'regression_energy_cleanLevel4_testy', 'regression_energy_cleanLevel4_trainX', 'regression_energy_cleanLevel4_trainy']
['forecasting_Hs2SwineFarmWithWeatherTime', 'forecasting_strawberryOpen', 'regression_energy']


### 1-2. Data Ingestion
#### 1-2-1. Select data name 

In [65]:
dataset_name = model_purpose + '_' + app_name  
data_name_X = dataset_name + '_cleanLevel' + str(data_clean_level)+'_'+step+'X'
data_name_y = dataset_name+'_cleanLevel' + str(data_clean_level)+'_'+ step+'y'
data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'ms_name':data_name_X})

In [66]:
data_meta

[{'bucket_name': 'integration',
  'collection_name': 'forecasting_Hs2SwineFarmWithWeatherTime',
  'ms_name': 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX',
  'ingestion_type': 'multiple_ms_by_time',
  'ingestion_param': {'start_time': '2021-02-01 00:00:00',
   'end_time': '2021-03-10 00:00:00',
   'ms_list_info': [['farm_swine_air', 'HS2'],
    ['weather_outdoor_keti_clean', 'sangju'],
    ['life_additional_Info', 'trigonometicInfoByHours']]},
  'processing_type': 'step_3',
  'process_param': {'refine_param': {'removeDuplication': {'flag': True},
    'staticFrequency': {'flag': True, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': True},
    'unCertainErrorToNaN': {'flag': True,
     'param': {'outlierDetectorConfig': [{'algorithm': 'IQR',
        'percentile': 99,
        'alg_parameter': {'weight': 100}}]}}},
   'imputation_param': {'flag': True,
    'imputation_method': [{'min': 0,
      'max': 2,
      'method': 'linear',
      'parameter': {}

#### 1-2-2. X-y Data Ingestion pipeline

In [67]:
# Data Ingestion
ingestion_method = 'ms_all'
ingestion_param_X = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_X,
    'feature_list' : feature_X_list                              
}
ingestion_param_y = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_y,
    'feature_list' : feature_y_list                              
}

In [68]:
data_X, data_y = ML_pipeline.Xy_data_preparation(ingestion_param_X, data_y_flag, ingestion_param_y, ingestion_method, db_client)

### 1-2-2. Random Nan Insert (Test)

In [69]:
nan_ratio = 0.00

In [70]:
data_X = tool.random_nan_df(data_X, nan_ratio)
data_y = tool.random_nan_df(data_y, nan_ratio)

../../../../Clust/clust/ML/common/tool.py:17: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df.loc[df.sample(frac=nan_ratio). index, col] = pd.np.nan
/home/leezy/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/leezy/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


#### 1-2-3. Data scaling

In [71]:
scaler_param='scale'
scale_method='minmax'
scaler_path = './scaler/'

In [72]:
dataX_scaled, X_scalerFilePath, datay_scaled, y_scalerFilePath= ML_pipeline.Xy_data_scaling_train(data_name_X, data_X, data_name_y, data_y, scaler_path, scaler_param, scale_method)

['Temperature', 'out_temp', 'sin_hour']
Make New scaler File
['Temperature']
Make New scaler File


## 2. Cleaning and split
### 2.1 pipeline - clean low quality column

In [73]:
model_clean = True
nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':100000}

In [74]:
dataX_scaled = ML_pipeline.clean_low_quality_column(model_clean, nan_process_info, dataX_scaled)
feature_X_list= list(dataX_scaled.columns)

3 ---> 3


### 2.2 Train/Val Split pipeline

In [75]:
split_ratio = 0.8

In [76]:
# TODO 데이터 나뉘는 부분 추가로 작성된 것 지수님에게 물어봐야 함
day_window_size = tool.get_default_day_window_size(dataX_scaled)
train_x, val_x, train_y, val_y = ML_pipeline.split_data_by_mode(split_mode, split_ratio, dataX_scaled, datay_scaled, day_window_size)

### 2.3 Data Transformation & Clean2 pipeline

In [77]:
max_nan_limit_ratio = 0.5
if split_mode =='windows_split':
    transformParameter = {
            'past_step':day_window_size,
            'max_nan_limit_ratio': max_nan_limit_ratio
    }
else:
    transformParameter = {
            'future_step': 2,
            'past_step': 12, 
            'max_nan_limit_ratio': max_nan_limit_ratio
    }

In [78]:
train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transformParameter, train_x, train_y)
val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transformParameter, val_x, val_y)

nan_limit_num:  6
Original num: 4258 Final num: 4176 NaN num: 82
nan_limit_num:  6
Original num: 1065 Final num: 1051 NaN num: 14


In [79]:
print(train_X_array.shape)
print(train_y_array.shape)
print(val_X_array.shape)
print(val_y_array.shape)

(4176, 12, 3)
(4176, 1)
(1051, 12, 3)
(1051, 1)


### 2.4 Set Model and train parameter

In [80]:
# RNN models (RNN, LSTM, GRU) parameters
seq_len, input_size = train_X_array.shape[1], train_X_array.shape[2]
model_method = 'GRU_rg' # Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 
if model_method == 'LSTM_rg' or model_method == 'GRU_rg':
    modelParameter = {
        'rnn_type': 'lstm',
        'input_size': input_size, 
        'hidden_size': 64,
        'num_layers': 2,
        'output_dim': 1, 
        'dropout': 0.1, 
        'bidirectional': True
    }
# CNN_1D model parameters
elif model_method == 'CNN_1D_rg':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'output_channels': 64,
    'kernel_size': 3,
    'stride': 1,
    'padding': 0, 
    'dropout': 0.1
    }
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_rg':
    modelParameter = {
    'input_size': input_size,
    'num_layers': 2,
    'lstm_dropout': 0.4,
    'fc_dropout': 0.1
    }
# FC model parameters
elif model_method == 'FC_rg':
    modelParameter = {
    'input_size': input_size,
    'dropout': 0.1,
    'bias': True
    }
    
train_parameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 20, 
    'batch_size': 16
}
model_name = None
model_file_path = None

### 2.5 Set Model name and path pipeline

In [81]:
# model_name and file path check and create
model_name = tool.get_default_model_name(model_name, app_name, model_method, model_clean)
model_file_path = tool.get_default_model_path(model_name, data_name_X, model_method, train_parameter)

./Models/GRU_rg/Hs2SwineFarmWithWeatherTime_GRU_rg_True/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX/e0c69926ea63923d2dbafa434a27060e/model.pkl


In [82]:
model_name

'Hs2SwineFarmWithWeatherTime_GRU_rg_True'

## 3. Training

In [20]:
ML_pipeline.CLUST_regresstion_train(train_parameter, model_method, modelParameter, model_file_path, train_X_array, train_y_array, val_X_array, val_y_array)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

## 4. save metadata

In [ ]:
# from Clust.clust.transformation.general.dataScaler import encode_hash_style
modelTags =["model_tag_example"]
trainDataType = "timeseries"

modelInfoMeta ={
    "trainDataInfo":data_meta,
    "modelName":model_name,
    "dataSplitMode":split_mode,
    "featureXList":feature_X_list,
    "featureyList":feature_y_list,
    "data_y_flag": data_y_flag,
    "trainDataType":trainDataType,
    "modelPurpose":model_purpose,
    "modelMethod":model_method,
    "modelTags":modelTags,
    "modelCleanLevel":model_clean,
    "trainParameter": train_parameter,
    "modelParameter": modelParameter,
    "transformParameter":transformParameter,
    "scalerParam":scaler_param,
    "trainDataName":[data_name_X, data_name_y], 

    "files":{
        "modelFile":{
            "fileName":"model.pth",
            "filePath":model_file_path
        },
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":y_scalerFilePath      
        }
    }
}

In [ ]:
from Clust.clust.ML.tool import meta as ml_meta
modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, modelInfoMeta)